## CCP-BioSim Training Course:
### Basic statistical methods in MD data analysis

When you are analysing MD trajectories frequently you are dealing with lots of data, and the data is noisy. Showing that measurements you make and conclusions you come to are statistically significant (or not!) is important.

This notebook illustrates the application of some basic ststistical methods to the analysis of simulation data:

1. Caculating means and confidence intervals.
2. Dealing with equilibration.

The dataset here is taken from the 1.03 millisecond ANTON BPTI dataset from Shaw et al. (http://science.sciencemag.org/content/330/6002/341.full), and is the distance between the N- and C-terminal atoms. Here we look at just the first 10 microseconds of this simulation, sampled every ten nanoseconds.

We begin by importing the key Python libraries: numpy and matplotlib:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline 
plt.rcParams.update({'font.size': 15}) #This sets a better default label size for plots

Next we load the distance data, and create a numpy array with the timepoint data which will be used later when we plot things:

In [ ]:
distances = np.load('data/end_to_end.npy')
n_samples = distances.size
times = np.arange(n_samples) * 0.01 # The time, in microseconds

Let's investigate a metric from this simulation: the average distance between the first and last C-alpha atoms.

To begin with let's imagine we ran the simulation for just ten microseconds so only have the first 1000 data points. In the next cell we cut the trajectory down to size, and then use MDTraj's `compute_distances()` method to find the end-to-end distance in each frame: 

In [ ]:
plt.plot(times, distances)
plt.xlabel(r'time ($\mu$sec)')
plt.ylabel('end-to-end distance (nm)')

So what's the mean end-to-end distance? Here's a very simple approach - calculate the mean, and because we know a little bit about statistics, we also calculate the standard error of the mean, and from this the classic 95% confidence interval. For this we make use of numpy's `mean()` and `std()` (standard deviation) methods:

In [ ]:
# The following four lines create lists we will use to store values we calculate as we work through this notebook.
d_means = [None] * 4
d_c95lows = [None] * 4
d_c95highs = [None] * 4
methods = [None] * 4

d_means[0] = distances.mean()
stderr = distances.std()/np.sqrt(n_samples)
d_c95lows[0] = d_means[0] - 1.96 * stderr # The lower bound of the 95% confidence interval
d_c95highs[0] = d_means[0] + 1.96 * stderr # The upper bound of the 95% confidence interval

text = "{}:\nMean end to end distance: {:3.3f} nm, 95% confidence interval {:3.3f} nm to {:3.3f} nm\n"
methods[0] = "Naive approach"
for i in range(1):
    print(text.format(methods[i], d_means[i], d_c95lows[i], d_c95highs[i]))

The above uses every value for the end-to-end distance. But is this valid? Only if the individual values are uncorrelated (see https://en.wikipedia.org/wiki/Unbiased_estimation_of_standard_deviation#Effect_of_autocorrelation_(serial_correlation)). Let's check for this.

The cell below calculates the autocorrelation function for the distances, and plots its first ten values:

In [ ]:
d2 = distances - distances.mean()
c = np.correlate(d2, d2, 'full')
c = c[int(c.size/2):]
c /= (c.var() * np.arange(d2.size, 0, -1))
plt.xlabel('sample interval')
plt.ylabel('correlation')
plt.plot(c[:10])

The autocorellation function only reaches zero after a time lag of about 4 samples (equivalent to 40 nanoseconds), so we should subsample the distance data at this interval to get truely uncorrelated values from which to calculate statistics.

This is what we do in the next cell, then recalculate the mean end-to-end distance:

In [ ]:
d_uncorr = distances[::4]
n_samples_uncorr = d_uncorr.size
time_uncorr = times[::4]
d_means[1] = d_uncorr.mean()
stderr = d_uncorr.std()/np.sqrt(n_samples_uncorr)
d_c95lows[1] = d_means[1] - stderr * 1.96
d_c95highs[1] = d_means[1] + stderr * 1.96
methods[1] = "After checking for correlation"
for i in range(2):
    print(text.format(methods[i], d_means[i], d_c95lows[i], d_c95highs[i]))

Note the new estimate of the mean end-to-end distance is considerably lower - below the 95% confidence limit in the original analysis.

Up to now we have implicitly been working under the assumtion that the data come from a normal distribution - but is this likely? Not really, after all, the aim of MD is to generate a Boltzmann weighted ansemble, not a normal one. If our data is not normally distributed then exactly what a "mean value" means, and what the confidence limits on it are, will need to be reconsidered.

In the cell below we plot a histogram of the distance values:

In [ ]:
H, edges = np.histogram(d_uncorr, bins='auto', density=True)
midpoints = [edges[i:i+2].mean() for i in range(len(edges) - 1)]
plt.xlabel('end-to-end distance')
plt.ylabel('probability density')
plt.plot(midpoints, H)

Yup -= not a normal distribution at all. In this situation it is better to use the method of bootstrapping https://en.wikipedia.org/wiki/Bootstrapping_(statistics) to estimate the confidence interval for the mean.

In [ ]:
b_means = []
for i in range(1000):
    b_means.append(np.random.choice(d_uncorr, size=n_samples_uncorr).mean())
b_means.sort()
d_means[2] = d_means[1] # our estimate of the mean is unchanged from last time, only the confidence interval
d_c95lows[2] = b_means[50] # because only 50/1000 (5%) of the estimated means are less than this
d_c95highs[2] = b_means[950] # because only 50/1000 (5%) of the estimated means are more than this
methods[2] = "With bootstrapping"
for i in range(3):
    print(text.format(methods[i], d_means[i], d_c95lows[i], d_c95highs[i]))

Note that by taking proper account of the non-normal distribution in the end-to-end distance samples, the 95% confidence range tightens up quite a bit.

OK, we have a mean and some confidence intervals. But is this a converged value? Is the simulation long enough? 

One simple approach is to say that it may be, if the mean end to end distance calculated from the first half of the simulation is close to the value calculated from the second half. Below we plot the difference between the means for the first and second halves of the simulation as a function of increasing total simulation length: i.e., comparing 0:100 with 100:200, then 0:200 with 200:400, etc. The expectation might be that as the sample sizes increase, the difference between the means will tend to zero:

In [ ]:
mean_length_difference = []
stepsize = int(len(d_uncorr)/100)
for i in range(stepsize, n_samples_uncorr + stepsize, stepsize):
    j = i // 2
    mean_length_difference.append(d_uncorr[:j].mean() - d_uncorr[j:i].mean())

plt.xlabel('simulation length (microsec.)')
plt.ylabel('difference in block means')
plt.plot(time_uncorr[1::stepsize], mean_length_difference)
plt.plot(time_uncorr[1::stepsize], np.zeros(len(mean_length_difference))) # add a line at y=0

Hmm - worryingly, the difference is tending to a positive number, what might be happening? One possibility os that we need to consider the process of equilibration. It might be that the early data from the simulation is unreliable (introduces a bias) because the system is still relaxing.

To test this, use a similar approach to the one above, only this time trim increasing amounts off the start of the dataset and compare the means of the two halves of the remainig data:

In [ ]:
mean_length_difference = []
mean_sem = []
for i in range(stepsize, n_samples_uncorr, stepsize):
    batch_size = (n_samples_uncorr - i) // 2
    j = i + batch_size
    mean_length_difference.append(d_uncorr[i:j].mean() - d_uncorr[j:].mean())
    mean_sem.append((d_uncorr[i:j].std() + d_uncorr[j:].std()) /(2 * np.sqrt(batch_size)))
plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.xlabel('discarded snapshots')
plt.ylabel('difference in block means')
plt.plot(mean_length_difference)
plt.plot(np.zeros(len(mean_length_difference))) # add a line at y=0
plt.subplot(122)
plt.xlabel('discarded snapshots')
plt.ylabel('average block sem')
plt.plot(mean_sem)

The left-hand plot shows that as we discard some data from the beginning of the simulation, the difference between the means for the first and second halves of the remaining data decreases, if somewhat erratically. But as we discard more data, the number of remaining samples decreases, and so the standard error of the result increases (see the right hand plot) and so our confidence in the difference between the means decreases. 

We have make some kind of trade-off here. Let's go for removing the first 60 values:

In [ ]:
d_uncorr_trimmed = d_uncorr[60:]
d_means[3] = d_uncorr_trimmed.mean()
b_means = []
for i in range(1000):
    b_means.append(np.random.choice(d_uncorr_trimmed, size=d_uncorr_trimmed.size).mean())
b_means.sort()
d_c95lows[3] = b_means[50]
d_c95highs[3] = b_means[950]
methods[3] = "After removing equilibration phase"
for i in range(4):
    print(text.format(methods[i], d_means[i], d_c95lows[i], d_c95highs[i]))

The estimate of the mean value has dropped quite a bit, and with it the confidence interval.

-----------------
### Summary

MD generates lots of data, so there is every reason to take full advantage of it and put your data analysis on a sound footing. You can see how an increasingly careful analysis of the data has a significant effect on the metrics extracted from it. 

Three simple questions are always worth asking:

1. Is there any evidence of equilibration artifacts?
2. Are my data values truely uncorrellated?
3. How is my data distributed - is it normal or not?